In [6]:
!pip install scikit-learn

   ---------------------------------------- 0.0/11.2 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.2 MB 3.6 MB/s eta 0:00:04
   - -------------------------------------- 0.5/11.2 MB 5.8 MB/s eta 0:00:02
   ---- ----------------------------------- 1.1/11.2 MB 9.0 MB/s eta 0:00:02
   ------- -------------------------------- 2.0/11.2 MB 11.6 MB/s eta 0:00:01
   ----------- ---------------------------- 3.2/11.2 MB 14.7 MB/s eta 0:00:01
   ----------------- ---------------------- 4.9/11.2 MB 18.3 MB/s eta 0:00:01
   ------------------------- -------------- 7.2/11.2 MB 21.8 MB/s eta 0:00:01
   ------------------------------------ --- 10.2/11.2 MB 27.2 MB/s eta 0:00:01
   ---------------------------------------  11.2/11.2 MB 38.6 MB/s eta 0:00:01
   ---------------------------------------- 11.2/11.2 MB 34.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/301.8 kB ? eta -:--:--
   --------------------------------------- 301.8/301.8 kB 18.2 MB/s eta 0:00:00

In [9]:
import os
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import importlib
import getpass
import glob
import seaborn as sns
import functions
import spectrogram_plotting_functions
import plotting_styles
import sklearn
from sklearn.decomposition import PCA, FastICA

import scipy.stats
import mne_connectivity
importlib.reload(functions) #loads our custom made functions.py file
importlib.reload(spectrogram_plotting_functions)
importlib.reload(plotting_styles)


<module 'plotting_styles' from 'c:\\Users\\sinha\\Documents\\GitHub\\cpl_analysis_naman\\plotting_styles.py'>

In [3]:
#Fetch the current user
user= (getpass.getuser())
print("Hello", user)

#Set the basepath, savepath and load the data files
base='C:\\Users\\{}\\Dropbox\\CPLab'.format(user)
files = glob.glob(base+'\\all_data_mat\\*.mat')
savepath = base+'\\results\\'
print(files)

all_bands_dict = {'total':[1,100], 'beta':[12,20], 'gamma':[30,80], 'theta':[4,12]}


Hello sinha
['C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat\\20230529_dk1_nocontext.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat\\20230529_dk3_nocontext.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat\\20230529_dk5_nocontext.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat\\20230529_dk6_nocontext.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat\\20230531_dk1_nocontext_day2.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat\\20230531_dk3_nocontext_day2.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat\\20230531_dk5_nocontext_day2.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat\\20230531_dk6_nocontext_day2.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat\\20230609_dk1_BW_nocontext_day1.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat\\20230609_dk3_BW_nocontext_day1.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat\\20230610_dk1_BW_nocontext_day2.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat\\20230610_dk3_BW_nocontext_day2.mat',

In [10]:

keyboard_dict={'98':'b','119':'w','120':'nc','49':'1','48':'0'} #specifying the map of keyboard annotations to their meanings.
all_bands={'total':[1,100],'beta':[12,30], 'gamma':[30,80], 'theta':[4,12]}

files=[f'C:\\Users\\{user}\\Dropbox\\CPLab\\all_data_mat_filtered\\20230615_dk6_BW_context_day1.mat', f'C:\\Users\\{user}\\Dropbox\\CPLab\\all_data_mat\\20230626_dk6_BW_nocontext_day1.mat'] #This is just for testing purposes

#Initializing a few empty things to store data
events_codes_all = {}
compiled_data_all_epochs = []
compiled_data_list=[]
baseline_lfp_all = []
normalization_comparison_all = []
for file in files: #Looping through data files
    
    ## Get the date, mouse_id and task from the file name
    base_name = os.path.basename(file)
    base_name, _ = os.path.splitext(base_name)
    date, mouse_id, task=functions.exp_params(base_name) #Using a custom made function [see functions.py]
    print(date, mouse_id, task)
    if task == 'nocontextday2' or task == 'nocontextos2':
        task = 'nocontext'
    if task =='nocontext':
        continue
    f=h5py.File(file, 'r')  ## Open the data file
    channels = list(f.keys()) ## Extract channels list from the data file
    print(channels)
    if not any("AON" in channel or "vHp" in channel for channel in channels):
        continue    
    events,reference_electrode=functions.get_keyboard_and_ref_channels(f,channels)

    events_codes=np.array(events['codes'][0]) #saving the keyboard annotations of the events (door open, door close etc.)
    events_times=np.array(events['times'][0]) #saving when the events happened
    events_codes_all[base_name] = events_codes #saving the codes in a dictionary to be analyzed later for events other than the ones in our keyboard_dict map
    
    #Generating epochs from events (epochs are basically start of a trial and end of a trial)
    epochs=functions.generate_epochs_with_first_event(events_codes, events_times)

    # task Start time
    first_event=events_times[0]
    #finding global start and end time of all channels, since they start and end recordings at different times
    global_start_time, global_end_time=functions.find_global_start_end_times(f,channels)
    
    ## Reference electrode finding and padding
    reference_time = np.array(reference_electrode['times']).flatten()
    reference_value = np.array(reference_electrode['values']).flatten()
    padd_ref_data,padded_ref_time=functions.pad_raw_data_raw_time(reference_value,reference_time,global_start_time,global_end_time,sampling_rate=2000)


    for channeli in channels:
        if "AON" in channeli or  "vHp" in channeli :

            # Extracting raw data and time
            data_all=f[channeli]
            raw_data=np.array(data_all['values']).flatten()
            raw_time = np.array(data_all['times']).flatten()
            sampling_rate = 2000
            print(channel_id)
            print(raw_data.shape, raw_time.shape, sampling_rate)
            
            padded_data,padded_time=functions.pad_raw_data_raw_time(raw_data,raw_time,global_start_time,global_end_time,sampling_rate)
            subtracted_data = padded_data - padd_ref_data
            raw_data=subtracted_data
            notch_filtered_data = functions.iir_notch(raw_data, sampling_rate, 60)
            ica=FastICA(n_components=6, random_state=0)
            ica.fit(notch_filtered_data.reshape(-1,1))
            ica_data=ica.fit_transform(notch_filtered_data.reshape(-1,1))
            #notch_filtered_data=ica_data.flatten()
            
            
#             data_before, time, baseline_mean, baseline_std=functions.baseline_data_normalization(notch_filtered_data, raw_time, first_event, sampling_rate)
#             first_event_index=np.where(raw_time>first_event)[0][0]

#             baseline_row=[mouse_id,task,channel_id,np.array(data_before)]
#             baseline_lfp_all.append(baseline_row)
#             normalized_data=notch_filtered_data

#             #Saving non-normalized data and normalized data for plotting
#             normalization_row=[mouse_id,task,channel_id,[notch_filtered_data[first_event_index:first_event_index+30*sampling_rate]],np.mean(data_before),np.std(data_before),[normalized_data[first_event_index:first_event_index+30*sampling_rate]]]
#             normalization_comparison_all.append(normalization_row)


#             for i,epochi in enumerate(epochs):
                
#                 compiled_data = pd.DataFrame() # Initializing a dataframe to store the data of a single epoch

#                 door_timestamp = epochi[0][0]
#                 trial_type = epochi[0][1]
#                 dig_type = epochi[1, 1]
#                 dig_timestamp = epochi[1, 0]
#                 print(door_timestamp,trial_type,dig_timestamp,dig_type)
                
                
#                 data_complete_trial=functions.extract_complete_trial_data(notch_filtered_data,time,door_timestamp,dig_timestamp,sampling_rate)
#                 data_trial_before, data_trial_after=functions.extract_door_data(notch_filtered_data,time,door_timestamp,sampling_rate)
#                 data_dig_before, data_dig_after=functions.extract_dig_data(notch_filtered_data,time,dig_timestamp,sampling_rate)
#                 epoch_data = [data_complete_trial, data_trial_before, data_trial_after, data_dig_before, data_dig_after]
#                 epoch_data = [functions.zscore_event_data(x, baseline_mean, baseline_std) for x in epoch_data]

#                 compiled_data = {
#                     'rat': mouse_id,
#                     'date': date,
#                     'task': task,
#                     'channel': channel_id,
#                     'trial': i,
#                     'timestamps': [door_timestamp, dig_timestamp],
#                     'side': keyboard_dict[str(int(trial_type))],
#                     'correct?': keyboard_dict[str(int(dig_type))],
#                     'first 30 seconds power': functions.calculate_power_1D(data_before),
#                     'time': time,
#                     'complete_trial': epoch_data[0],
#                     'pre_door': epoch_data[1],
#                     'post_door': epoch_data[2],
#                     'pre_dig': epoch_data[3],
#                     'post_dig': epoch_data[4]
#                 }

#                 compiled_data_list.append(compiled_data)

# compiled_data_all_epochs.extend(compiled_data_list)
# compiled_data_all_epochs = pd.DataFrame(compiled_data_all_epochs)
# compiled_data_all_epochs= compiled_data_all_epochs[compiled_data_all_epochs['task']!='nocontext']
# baseline_lfp_all = pd.DataFrame(baseline_lfp_all, columns=['rat', 'task', 'channel', 'data'])
# normalization_comparison_all = pd.DataFrame(normalization_comparison_all, columns=['rat', 'task', 'channel', 'non_normalized_data', 'baseline_mean', 'baseline_std', 'normalized_data'])   


20230615 dk6 BWcontext
['Keyboard', 'LFP1_vHp', 'LFP2_vHp', 'LFP3_AON', 'LFP4_AON', 'Ref', 'Respirat', 'file']
LFP3_AON
(2695355,) (2695355,) 2000
notch filter applied


c:\Users\sinha\anaconda3\envs\lfp\lib\site-packages\sklearn\decomposition\_fastica.py:598: UserWarning: n_components is too large: it will be set to 1
  warnings.warn(
c:\Users\sinha\anaconda3\envs\lfp\lib\site-packages\sklearn\decomposition\_fastica.py:598: UserWarning: n_components is too large: it will be set to 1
  warnings.warn(


LFP3_AON
(2695355,) (2695355,) 2000
notch filter applied


c:\Users\sinha\anaconda3\envs\lfp\lib\site-packages\sklearn\decomposition\_fastica.py:598: UserWarning: n_components is too large: it will be set to 1
  warnings.warn(
c:\Users\sinha\anaconda3\envs\lfp\lib\site-packages\sklearn\decomposition\_fastica.py:598: UserWarning: n_components is too large: it will be set to 1
  warnings.warn(


LFP3_AON
(2695356,) (2695356,) 2000
notch filter applied


...

c:\Users\sinha\anaconda3\envs\lfp\lib\site-packages\sklearn\decomposition\_fastica.py:598: UserWarning: n_components is too large: it will be set to 1
  warnings.warn(
c:\Users\sinha\anaconda3\envs\lfp\lib\site-packages\sklearn\decomposition\_fastica.py:598: UserWarning: n_components is too large: it will be set to 1
  warnings.warn(


LFP3_AON
(2070661,) (2070661,) 2000
notch filter applied


c:\Users\sinha\anaconda3\envs\lfp\lib\site-packages\sklearn\decomposition\_fastica.py:598: UserWarning: n_components is too large: it will be set to 1
  warnings.warn(
c:\Users\sinha\anaconda3\envs\lfp\lib\site-packages\sklearn\decomposition\_fastica.py:598: UserWarning: n_components is too large: it will be set to 1
  warnings.warn(


LFP3_AON
(2070661,) (2070661,) 2000
notch filter applied


c:\Users\sinha\anaconda3\envs\lfp\lib\site-packages\sklearn\decomposition\_fastica.py:598: UserWarning: n_components is too large: it will be set to 1
  warnings.warn(
c:\Users\sinha\anaconda3\envs\lfp\lib\site-packages\sklearn\decomposition\_fastica.py:598: UserWarning: n_components is too large: it will be set to 1
  warnings.warn(
